> conv提特征 向量点积往大收敛 提取的特证逐渐与原图相似

In [4]:
# demo
import torch
import torch.nn as nn
import torch.nn.functional as F
in_channels = 1
out_channels = 1
kernel_size = 3 #传入元组 卷积核可以为矩形
batch_size = 2
bias = False
input_size = [batch_size, in_channels, 4, 4]
# 实例化类 首字母大写
conv_layer_test = torch.nn.Conv2d(3, 9, kernel_size, bias=bias)
# padding = sample
conv_layer = torch.nn.Conv2d(1, 1, kernel_size, bias=bias)
print(conv_layer.weight)
x = torch.randn(input_size)
x = F.dropout(x)
x = conv_layer(x)
print(conv_layer_test.weight.shape)
print(x)

Parameter containing:
tensor([[[[-0.3081,  0.1996,  0.1196],
          [-0.2311, -0.0334,  0.2912],
          [ 0.1883,  0.1224,  0.1053]]]], requires_grad=True)
torch.Size([9, 3, 3, 3])
tensor([[[[-0.6239, -1.4214],
          [ 1.0948, -0.4845]]],


        [[[ 1.1987, -0.5851],
          [ 1.0835, -1.0004]]]], grad_fn=<MkldnnConvolutionBackward>)


> 函数实现

In [5]:
# 用原始的矩阵运算实现二维卷积,先不考虑batchsize维度和channel维数
def matrix_multiplication_for_conv2d(input, kernel, bias=0, stride=1, padding=0):
    if padding > 0:
        input = F.pad(input, (padding, padding, padding, padding))
    input_h, input_w = input.shape
    kernel_h, kernel_w = kernel.shape
    print(input.size(0))
    output_w = int(((input_w - kernel_w)/stride + 1))
    output_h = int(((input_h - kernel_h)/stride + 1))
    print(output_w)
    output = torch.zeros(output_w, output_h)
    for i in range(0, output_h, stride):
        for j in range(0, output_w, stride):
            # 方法1
            # p1d = (i, input_w -kernel_w-i,stride*i, input_h -kernel_h-stride*i)
            # kernel_pad = F.pad(kernel, p1d, 'constant', 0)
            # # print(kernel_pad)
            # output[int(i/stride), int(j/stride)] = torch.sum(torch.mul(kernel_pad, input))、
            # 方法2
            region = input[i:i+kernel_h,j:j+kernel_w]
            output[int(i/stride), int(j/stride)] = torch.sum(region * kernel)+bias
    return output
bias = torch.randn(1)
x = torch.randn(6, 6)
kernel = torch.randn(3, 3)
output = matrix_multiplication_for_conv2d(x, kernel,bias=bias)
print(output)
pytorch_api_output = F.conv2d(x.reshape(1,1,x.size(0),x.size(1)),kernel.reshape(1,1,kernel.size(0),kernel.size(1)))
print(pytorch_api_output)

6
4
tensor([[-3.6580, -0.9842,  3.4693,  1.8781],
        [ 3.9370, -0.3824,  0.7045, -2.0984],
        [-0.3617,  1.8660, -2.1826, -0.2953],
        [-3.0925, -0.4673,  2.7907, -1.5380]])
tensor([[[[-3.8849, -1.2110,  3.2424,  1.6512],
          [ 3.7101, -0.6093,  0.4777, -2.3252],
          [-0.5886,  1.6391, -2.4095, -0.5222],
          [-3.3194, -0.6941,  2.5638, -1.7648]]]])


> flatten 版本

In [6]:
# 用原始的矩阵运算实现二维卷积,先不考虑batchsize维度和channel维数,flatten版本 flatten作用把张量拉直成1维
def matrix_multiplication_for_conv2d_flatten(input, kernel, bias=0, stride=1, padding=0):
    if padding > 0:
        input = F.pad(input, (padding, padding, padding, padding))
    input_h, input_w = input.shape
    kernel_h, kernel_w = kernel.shape
    print(input.size(0))
    output_w = int(((input_w - kernel_w)/stride + 1))
    output_h = int(((input_h - kernel_h)/stride + 1))
    print(output_w)
    output = torch.zeros(output_w, output_h)
    region_matrix = torch.zeros(output.numel(), kernel.numel())
    kernel_matrix = kernel.reshape((kernel.numel(), 1))
    
    for i in range(0, output_h-kernel_h+1, stride):
        for j in range(0, output_w-kernel_w+1, stride):
            region = input[i:i+kernel_h, j:j+kernel_w] 
            region_vector = torch.flatten(region)
            region_matrix[i+j] = region_vector
    output_matrix = region_matrix @ kernel_matrix
    output = output_matrix.reshape((output_h, output_w))
    return output
bias = torch.randn(1)
x = torch.randn(6, 6)
kernel = torch.randn(3, 3)
output = matrix_multiplication_for_conv2d_flatten(x, kernel,bias=bias)
print(output)
pytorch_api_output = F.conv2d(x.reshape(1,1,x.size(0),x.size(1)),kernel.reshape(1,1,kernel.size(0),kernel.size(1)))
print(pytorch_api_output)

6
4
tensor([[-0.5224,  0.0477,  2.1909,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000]])
tensor([[[[-0.5224, -0.0982, -1.9555, -0.1725],
          [ 0.0477,  2.1909,  2.0212, -0.9861],
          [-1.6592, -1.7292,  2.4450,  1.0094],
          [ 0.2891,  0.1795, -1.9192,  0.3684]]]])
